In [1]:
import os
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.datasets import mnist
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

from keras.models import load_model
from keras.preprocessing import image

from google.colab import files
import numpy as np

import numpy as np
%matplotlib inline

# Подготовка данных для обучения сети

In [2]:
batch_size = 100 # Размер мини-выборки
nb_classes = 10 # Количество классов изображений
nb_epoch = 25 # Количество эпох для обучения
img_rows, img_cols = 28, 28 # Размер изображений

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


# Преобразование размерности и нормализация данных
Значение интенсивности пикселей в изображении находится в интервале [0,255]. Для наших целей их необходимо нормализовать - привести к значениям в интервале [0,1].

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

Преобразуем метки классов в удобный для обучения нейронной сети формат (one hot encoding)


In [5]:
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

#Конструируем нейронную сеть и компилируем:
loss (функция потерь) - оценка желаемого значения относительно спрогнозированного, применим categorical_crossentropy (кроссэнтропию) - функцию штрафа, которую следует использовать для задач классификации, как у нас;
optimizer (функция оптимизации) - агоритм "подгонки" внутренних параметров (весов и смещений) модели для минимизации функции потерь, optimizer="adam". Метод вычисляет индивидуальные адаптивные скорости обучения для различных параметров из оценок первого и второго моментов градиентов. Название получено из adaptive moment estimation - адаптивной оценки момента. Есть другие варианты оптимизаторов;
metrics (метрики) - используются для мониторинга процесса тренировки и тестирования, metrics=['accuracy'] значит, что мы будем вычислять в модели не только функцию штрафа, но и число правильно классифицированных примеров

In [84]:
print("Если у вас уже есть модель ее можно загрузить, если нет нажмите 'cancel upload'.\n")
mod = files.upload()

Если у вас уже есть модель ее можно загрузить, если нет нажмите 'cancel upload'.



Saving my_model.h5 to my_model.h5


In [85]:
pt = !ls
if pt[0].find('my_model.h5') != -1:

  model = load_model('my_model.h5')

else:

  # Создаем последовательную модель нейронной сети
  model = Sequential()
  # Первый сверточный слой
  model.add(Conv2D(img_rows, (3, 3), padding='same',
                          input_shape=(img_rows, img_cols, 1), activation='relu'))
  # Второй сверточный слой
  model.add(Conv2D(img_rows, (3, 3), activation='relu', padding='same'))
  # Первый слой подвыборки
  model.add(MaxPooling2D(pool_size=(2, 2)))
  # Слой регуляризации Dropout
  model.add(Dropout(0.25))

  # Третий сверточный слой
  model.add(Conv2D(2 * img_rows, (3, 3), padding='same', activation='relu'))
  # Четвертый сверточный слой
  model.add(Conv2D(2 * img_rows, (3, 3), activation='relu'))
  # Второй слой подвыборки
  model.add(MaxPooling2D(pool_size=(2, 2)))
  # Слой регуляризации Dropout
  model.add(Dropout(0.25))
  # Слой преобразования данных из 2D представления в плоское
  model.add(Flatten())
  # Полносвязный слой для классификации
  model.add(Dense(8 * img_rows, activation='relu'))
  # Слой регуляризации Dropout
  model.add(Dropout(0.5))
  # Выходной полносвязный слой
  model.add(Dense(nb_classes, activation='softmax'))

  #________________________________________________
  model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  print(model.summary())

  #Обучаем модель
  model.fit(x_train,y_train,epochs=10)
  #Сохраняем модель
  model.save('my_model.h5')
  #Для удаления старой модели:
  #del model
  #Для загрузки модели:
  #model = load_model('my_model.h5')

In [86]:
#Простая оценка
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0203 - accuracy: 0.9924 - 1s/epoch - 3ms/step


[0.020323092117905617, 0.9923999905586243]

In [9]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
from copy import deepcopy

In [56]:
def count_spot_stack(image, x, y): #количественная оценка "пятна"
  mx, my = image.shape
  xl, xr, yl, yr = x, x, y, y
  stack={(x, y)}
  res=1
  while stack:
    pixel=stack.pop()
    if image[pixel]==0:
      x, y = pixel
      res+=1

      if x < xl:
        xl = x
      elif x > xr:
        xr = x
      if y < yl:
        yl = y
      elif y > yr:
        yr = y

      stack|=set(pix for pix in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)] if (0<=pix[0]<mx and 0<=pix[1]<my))
      image[pixel]=255

  del stack
  return (xl, xr, yl, yr)

def find_spots(image): #Поиск цифр на изображении
  for y in range(image.shape[1]):
    for x in range(image.shape[0]):
      if image[x, y]==0:
        mask=deepcopy(image)
        xl, xr, yl, yr = count_spot_stack(image, x, y)
        xx=((xr-xl+1)//28)*3
        yy=((yr-yl+1)//28)*3
        xl=max(xl-xx, 0)
        xr+=xx
        yl=max(yl-yy, 0)
        yr+=yy
        mask^=~image
        yield deepcopy(mask[xl:xr+1, yl:yr+1])
        del mask

porog=255-75


In [79]:
img_load = files.upload()
for i in img_load.keys():
  pass

Saving изображение_2023-11-01_131544532.png to изображение_2023-11-01_131544532.png


In [87]:
img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)


img2 = deepcopy(img)
img2[img2<=porog]=0
img2[img2>porog]=255
img2.dtype=np.uint8

result=""
for spot in find_spots(deepcopy(img2)):
  temp=cv2.resize(spot, (28,28), interpolation = cv2.INTER_AREA)
  temp=temp.reshape(1,28,28,1)
  temp=1-temp/255
  prediction = model.predict([temp], verbose=0)[0]
  result+=str(np.argmax(prediction))

print(result)

0123456789
